In [1]:
import pandas as pd
import numpy as np


In [6]:
numerical = pd.read_csv('files_for_lab/numerical.csv')
categorical = pd.read_csv('files_for_lab/categorical.csv')
targets = pd.read_csv('files_for_lab/target.csv')

In [8]:
data = pd.concat([numerical, categorical, targets], axis = 1)
data = data.drop([83217,93811], axis=0)
category_1 = data[data['TARGET_B']==1].sample(len(data[data['TARGET_B']==0]), replace=True)
category_0 = data[data['TARGET_B']== 0 ]
data = pd.concat([category_0, category_1], axis = 0)
data = data.sample(frac =1) 
data = data.reset_index(drop=True)

In [10]:
y = data['TARGET_B']
X = data.drop(['TARGET_B'], axis = 1)

numericalX = X.select_dtypes(np.number)
categoricalX = X.select_dtypes(object)

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(drop='first').fit(categoricalX)
encoded_categorical = encoder.transform(categoricalX).toarray()
encoded_categorical = pd.DataFrame(encoded_categorical)
X = pd.concat([numericalX, encoded_categorical], axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

y_train_regression = X_train['TARGET_D']
y_test_regression = X_test['TARGET_D']

X_train = X_train.drop(['TARGET_D'], axis = 1)
X_test = X_test.drop(['TARGET_D'], axis = 1)

KeyError: 'TARGET_D'

In [13]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

0.6143940596382508
0.6040246225191156


In [ ]:
# downscaling gives back a higher score

In [14]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(max_depth=5,
                             min_samples_split=20,
                             min_samples_leaf =20)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=10)
print(np.mean(cross_val_scores))

0.6132140108000688


In [15]:
cross_val_scores

array([0.61058588, 0.60941274, 0.62114416, 0.61017183, 0.61348423,
       0.61120696, 0.61382927, 0.61559696, 0.61090407, 0.615804  ])

In [ ]:
# a false positive mean the business is spending money in mailing needlessly
# but a false negative would mean missing out entirely on a donation which is worse as long as donations are higher than the cost of mail
